<a href="https://colab.research.google.com/github/ManojKumarKolli/ManojKumar_INFO5731_Spring2024/blob/main/Assignments/Kolli_ManojKumar_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_reviews(movie_id, max_reviews=1000):
    # Initialize the empty list
    reviews = []
    start = 0  # Initial start index
    # imdb url
    base_url = f"https://www.imdb.com/title/{movie_id}/reviews/_ajax?ref_=undefined&paginationKey="
    last_key = ""
    # we will run this loop until we get the reviews
    while len(reviews) < max_reviews:
        response = requests.get(f"{base_url}{last_key}")
        if response.status_code != 200:
            break  # Break the loop if the request failed
        # read the html content
        soup = BeautifulSoup(response.content, 'html.parser')
        # getting the desired tags
        review_elements = soup.find_all("div", class_="text show-more__control")
        # we will go through every review
        for element in review_elements:
            reviews.append(element.get_text(strip=True))
            if len(reviews) >= max_reviews:
                break  # Stop if we reach the desired number of reviews

        # IMDb uses a pagination key for loading more reviews, which we need to update
        load_more_key = soup.find("div", class_="load-more-data")
        if load_more_key:
            last_key = load_more_key["data-key"]
        else:
            break  # No more reviews to load or no pagination key found

    return reviews[:max_reviews]

movie_id = "tt4154796"  # Avengers: Endgame
reviews = fetch_reviews(movie_id, 1000)

# Saving to CSV
df = pd.DataFrame(reviews, columns=["Review"])
df.to_csv("imdb_reviews.csv", index=False)

print(len(reviews), "reviews collected.")


1000 reviews collected.


In [3]:
df.shape

(1000, 1)

In [4]:
print(len(set(reviews)))

1000


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [5]:
# Write code for each of the sub parts with proper comments.

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

# NLTK data
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:

# Loading the reviews
df = pd.read_csv("imdb_reviews.csv")

# Function to remove noise (special characters and punctuations)
def remove_noise(text):
    return re.sub(r'[^\w\s]', '', str(text))

# Function to remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# Function to convert text to lowercase
def to_lowercase(text):
    return text.lower()

# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    return ' '.join([word for word in tokens if word not in stop_words])

# Function for stemming
def apply_stemming(text):
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)
    return ' '.join([stemmer.stem(word) for word in tokens])

# Function for lemmatization
def apply_lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in tokens])



In [7]:
# Step 1: Remove Noise
df['Clean_Review'] = df['Review'].apply(remove_noise)
print("After Removing Noise:")
print(df)


After Removing Noise:
                                                Review  \
0    But its a pretty good film. A bit of a mess in...   
1    Rating: 8.6Not as good as Infinity war pacing-...   
2    So it all ends. I have loved most of the MCU, ...   
3    This film is an emotional rollercoaster with s...   
4    After watching Infinity war, I was looking for...   
..                                                 ...   
995  It did its job at providing a satisfying concl...   
996  The last time he met, he knew she was staying ...   
997  Will probably get killed with dislikes, but he...   
998  OK they did all this work to not mess up the t...   
999  Everybody is here, Stan Lee even gets one more...   

                                          Clean_Review  
0    But its a pretty good film A bit of a mess in ...  
1    Rating 86Not as good as Infinity war pacingwis...  
2    So it all ends I have loved most of the MCU so...  
3    This film is an emotional rollercoaster with s..

In [8]:
# Step 2: Remove Numbers
df['Clean_Review'] = df['Clean_Review'].apply(remove_numbers)
print("\nAfter Removing Numbers:")
print(df)




After Removing Numbers:
                                                Review  \
0    But its a pretty good film. A bit of a mess in...   
1    Rating: 8.6Not as good as Infinity war pacing-...   
2    So it all ends. I have loved most of the MCU, ...   
3    This film is an emotional rollercoaster with s...   
4    After watching Infinity war, I was looking for...   
..                                                 ...   
995  It did its job at providing a satisfying concl...   
996  The last time he met, he knew she was staying ...   
997  Will probably get killed with dislikes, but he...   
998  OK they did all this work to not mess up the t...   
999  Everybody is here, Stan Lee even gets one more...   

                                          Clean_Review  
0    But its a pretty good film A bit of a mess in ...  
1    Rating Not as good as Infinity war pacingwise ...  
2    So it all ends I have loved most of the MCU so...  
3    This film is an emotional rollercoaster with 

In [9]:
# Step 3: Convert to Lowercase
df['Clean_Review'] = df['Clean_Review'].apply(to_lowercase)
print("\nAfter Converting to Lowercase:")
print(df)



After Converting to Lowercase:
                                                Review  \
0    But its a pretty good film. A bit of a mess in...   
1    Rating: 8.6Not as good as Infinity war pacing-...   
2    So it all ends. I have loved most of the MCU, ...   
3    This film is an emotional rollercoaster with s...   
4    After watching Infinity war, I was looking for...   
..                                                 ...   
995  It did its job at providing a satisfying concl...   
996  The last time he met, he knew she was staying ...   
997  Will probably get killed with dislikes, but he...   
998  OK they did all this work to not mess up the t...   
999  Everybody is here, Stan Lee even gets one more...   

                                          Clean_Review  
0    but its a pretty good film a bit of a mess in ...  
1    rating not as good as infinity war pacingwise ...  
2    so it all ends i have loved most of the mcu so...  
3    this film is an emotional rollercoaste

In [10]:
# Step 4: Remove Stopwords
df['Clean_Review'] = df['Clean_Review'].apply(remove_stopwords)
print("\nAfter Removing Stopwords:")
print(df)




After Removing Stopwords:
                                                Review  \
0    But its a pretty good film. A bit of a mess in...   
1    Rating: 8.6Not as good as Infinity war pacing-...   
2    So it all ends. I have loved most of the MCU, ...   
3    This film is an emotional rollercoaster with s...   
4    After watching Infinity war, I was looking for...   
..                                                 ...   
995  It did its job at providing a satisfying concl...   
996  The last time he met, he knew she was staying ...   
997  Will probably get killed with dislikes, but he...   
998  OK they did all this work to not mess up the t...   
999  Everybody is here, Stan Lee even gets one more...   

                                          Clean_Review  
0    pretty good film bit mess parts lacking cohesi...  
1    rating good infinity war pacingwise satisfying...  
2    ends loved mcu predictable awaited iw see woul...  
3    film emotional rollercoaster coolest superh

In [11]:
# # Step 5: Apply Stemming
# df['Clean_Review'] = df['Clean_Review'].apply(apply_stemming)
# print("\nAfter Stemming:")
# print(df)

# Here I am using Lemmatization

# Step 6: Apply Lemmatization
df['Clean_Review'] = df['Clean_Review'].apply(apply_lemmatization)
print("\nAfter Lemmatization:")
print(df)


After Lemmatization:
                                                Review  \
0    But its a pretty good film. A bit of a mess in...   
1    Rating: 8.6Not as good as Infinity war pacing-...   
2    So it all ends. I have loved most of the MCU, ...   
3    This film is an emotional rollercoaster with s...   
4    After watching Infinity war, I was looking for...   
..                                                 ...   
995  It did its job at providing a satisfying concl...   
996  The last time he met, he knew she was staying ...   
997  Will probably get killed with dislikes, but he...   
998  OK they did all this work to not mess up the t...   
999  Everybody is here, Stan Lee even gets one more...   

                                          Clean_Review  
0    pretty good film bit mess part lacking cohesiv...  
1    rating good infinity war pacingwise satisfying...  
2    end loved mcu predictable awaited iw see would...  
3    film emotional rollercoaster coolest superhero..

In [12]:
# Save cleaned dataset
df.to_csv("Cleaned_dataset.csv", index = False)

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [13]:
# Your code here
# (1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
def pos_tagging_and_counting(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Perform POS tagging
    tagged = pos_tag(tokens)

    # Initialize counters for each part of speech of interest
    pos_counts = Counter()

    # Define POS tags of interest
    pos_interest = {'NN': 'Noun', 'VB': 'Verb', 'JJ': 'Adjective', 'RB': 'Adverb'}

    # Count occurrences of each POS of interest
    for word, tag in tagged:
        if tag[:2] in pos_interest:
            pos_counts[pos_interest[tag[:2]]] += 1

    return pos_counts


In [15]:
# Apply the POS tagging and counting function to each review and collect results
pos_counts_list = df['Review'].apply(pos_tagging_and_counting).tolist()


In [16]:
# Create a new DataFrame from the POS counts list
pos_df = pd.DataFrame({
    'Nouns': [counts.get('Noun', 0) for counts in pos_counts_list],
    'Verbs': [counts.get('Verb', 0) for counts in pos_counts_list],
    'Adjectives': [counts.get('Adjective', 0) for counts in pos_counts_list],
    'Adverbs': [counts.get('Adverb', 0) for counts in pos_counts_list],
})

# take a new dataframe
pos_df['Review'] = df['Review'].values

# Display the new DataFrame to review the POS tagging results
print(pos_df.head())


   Nouns  Verbs  Adjectives  Adverbs  \
0    115     75          47       28   
1     15     10           8        3   
2     32     35          21       18   
3     11     12           6        7   
4      3      4           1        4   

                                              Review  
0  But its a pretty good film. A bit of a mess in...  
1  Rating: 8.6Not as good as Infinity war pacing-...  
2  So it all ends. I have loved most of the MCU, ...  
3  This film is an emotional rollercoaster with s...  
4  After watching Infinity war, I was looking for...  


In [17]:
# Aggregate counts for each POS
total_nouns = sum(counts.get('Noun', 0) for counts in pos_counts_list)
total_verbs = sum(counts.get('Verb', 0) for counts in pos_counts_list)
total_adjectives = sum(counts.get('Adjective', 0) for counts in pos_counts_list)
total_adverbs = sum(counts.get('Adverb', 0) for counts in pos_counts_list)

# Print the total counts
print(f"Total Nouns: {total_nouns}")
print(f"Total Verbs: {total_verbs}")
print(f"Total Adjectives: {total_adjectives}")
print(f"Total Adverbs: {total_adverbs}")

Total Nouns: 46397
Total Verbs: 32958
Total Adjectives: 15117
Total Adverbs: 13910


In [18]:
# (2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

import spacy
from spacy import displacy

# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

# Get the first sentence from the DataFrame
text = df.iloc[0]['Clean_Review']

# Plot the dependency graph
doc = nlp(text)
displacy.render(doc, style='dep', jupyter=True)

# Print token information
for token in doc:
    print(token.text, "-->", token.dep_, "-->", token.pos_)

pretty --> advmod --> ADV
good --> amod --> ADJ
film --> compound --> NOUN
bit --> compound --> NOUN
mess --> compound --> NOUN
part --> nsubj --> NOUN
lacking --> acl --> VERB
cohesive --> amod --> ADJ
effortless --> dobj --> NOUN
feel --> ROOT --> VERB
infinity --> compound --> NOUN
war --> nsubj --> NOUN
somehow --> advmod --> ADV
managed --> amod --> VERB
accomplish --> amod --> ADJ
silly --> amod --> ADJ
plot --> compound --> NOUN
hole --> compound --> NOUN
character --> nsubj --> NOUN
could --> aux --> AUX
ve --> aux --> AUX
cut --> ccomp --> VERB
ahem --> compound --> NOUN
captain --> compound --> NOUN
marvel --> compound --> NOUN
thanos --> nsubj --> NOUN
use --> ccomp --> VERB
captain --> compound --> NOUN
marvel --> compound --> NOUN
film --> dobj --> NOUN
ridiculous --> ROOT --> ADJ
she --> nsubj --> PRON
s --> aux --> AUX
start --> ccomp --> VERB
bail --> compound --> NOUN
reason --> dobj --> NOUN
pop --> compound --> NOUN
end --> nsubj --> NOUN
serve --> conj --> VERB
purp

In [19]:
# (3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

import spacy
from collections import Counter

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def named_entity_recognition(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Extract entities and their labels
    entities = [(ent.text.strip(), ent.label_) for ent in doc.ents]
    return entities

# Apply NER to each review and aggregate results
entity_lists = df['Review'].apply(named_entity_recognition)

# Flatten the list of entities from all reviews and count each unique entity
all_entities = [entity for sublist in entity_lists for entity in sublist]
entity_counts = Counter(all_entities)

# Print the total counts for each entity type
print("Entity counts:", entity_counts)



Entity counts: Counter({('Thanos', 'ORG'): 484, ('Avengers', 'NORP'): 443, ('MCU', 'ORG'): 293, ('Infinity War', 'EVENT'): 258, ('one', 'CARDINAL'): 251, ('Marvel', 'PERSON'): 215, ('first', 'ORDINAL'): 203, ('Hulk', 'ORG'): 143, ('Marvel', 'ORG'): 132, ('Thor', 'PERSON'): 114, ('Tony', 'PERSON'): 113, ('Iron Man', 'PERSON'): 109, ('Cap', 'PERSON'): 106, ('Thor', 'GPE'): 97, ('two', 'CARDINAL'): 95, ('half', 'CARDINAL'): 87, ('Nebula', 'PERSON'): 80, ('Marvel', 'GPE'): 73, ('3', 'CARDINAL'): 69, ('second', 'ORDINAL'): 68, ('3 hours', 'TIME'): 68, ('2', 'CARDINAL'): 64, ('Tony Stark', 'PERSON'): 61, ('Endgame', 'PERSON'): 57, ('Stark', 'PERSON'): 54, ('22', 'CARDINAL'): 52, ('CGI', 'ORG'): 51, ('1', 'CARDINAL'): 50, ('Hawkeye', 'GPE'): 45, ('10', 'CARDINAL'): 44, ('America', 'GPE'): 42, ('Black Widow', 'WORK_OF_ART'): 41, ('One', 'CARDINAL'): 40, ('First', 'ORDINAL'): 40, ('11 years', 'DATE'): 39, ('Infinity', 'NORP'): 37, ('Chris Evans', 'PERSON'): 35, ('Captain Marvel', 'ORG'): 35, ('

In [20]:
# Separate entities by type for counting
entity_types = {}
for entity, label in all_entities:
    if label not in entity_types:
        entity_types[label] = Counter()
    entity_types[label][entity] += 1

# Print counts by entity type
for label, entities in entity_types.items():
    print(f"Entity Type: {label}")
    for entity, count in entities.most_common():
        print(f"  {entity}: {count}")


Entity Type: ORG
  Thanos: 484
  MCU: 293
  Hulk: 143
  Marvel: 132
  CGI: 51
  Captain Marvel: 35
  Disney: 22
  the Russo Brothers: 20
  Banner: 19
  Falcon: 17
  Pym: 15
  Rings: 15
  Marvel Studios: 14
  Universe: 14
  Vision: 13
  Thor: 13
  Stones: 12
  The Russo Brothers: 12
  Galaxy: 11
  Scarlet Witch: 10
  Pepper: 10
  Marvel Cinematic: 10
  Clint: 10
  Infinity Stones: 8
  Natasha: 7
  Marvel Universe: 7
  VFX: 7
  Quantum: 6
  Renner: 6
  quantum: 6
  REALLY: 5
  IG: 5
  the Soul Stone: 5
  the Infinity Gauntlet: 5
  Antman: 5
  The Winter Soldier: 5
  Pepper Potts: 5
  Russo Brothers: 5
  the Quantum Realm: 5
  SHIELD: 5
  Gauntlet: 5
  Capt Marvel: 4
  Movie: 4
  the Marvel Cinematic: 4
  Bucky: 4
  Odin: 4
  Nebula: 4
  GOTG: 4
  PS: 4
  Spider-Man: 4
  Time: 4
  IMO: 4
  Korg: 4
  Fortnite: 4
  RDJ: 4
  War Machine: 4
  the Infinity Saga: 4
  Tesseract: 4
  the Space Stone: 4
  Banner/Hulk: 3
  Easter: 3
  Plot: 3
  Time Travel: 3
  Infinity Gauntlet: 3
  Infinity War's

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [21]:
# Write your response below
'''
It is always challenging to scrap data from the dynamic webpages. We need to work a lot on this.
We also need to take a choice between stemming and lemmatization based on the situation. There is a enough time to
complete this assignment.
'''